In [18]:
project_id = 'elife-data-pipeline'
dataset = 'de_dev'
view_name='v_data_science_editor_pubmed_ids'
mview_name='mv_data_science_editor_pubmed_ids'
query_name ='v_data_science_editor_pubmed_ids.sql'

In [19]:
import logging

from bigquery_views_manager.update_views import update_or_create_view
from bigquery_views_manager.materialize_views import materialize_view

from data_science_pipeline.utils.bq import get_client
from data_science_pipeline.sql import get_sql

In [3]:
LOGGER = logging.getLogger(__name__)
logging.basicConfig(level='INFO')

In [7]:
client = get_client(project_id=project_id)

In [21]:
LOGGER.info('processing: %s', query_name)
view_query = get_sql(query_name).format(
        project=project_id,
        dataset=dataset
)
update_or_create_view(
        client=client,
        view_name=view_name,
        view_query=view_query,
        dataset=dataset
)    

INFO:__main__:processing: v_data_science_editor_pubmed_ids.sql
INFO:bigquery_views_manager.update_views:updated or replaced view: elife-data-pipeline:de_dev.v_data_science_editor_pubmed_ids


In [22]:
materialize_view(
    client=client,
    source_view_name=view_name,
    destination_table_name=mview_name,
    project=project_id,
    source_dataset=dataset,
    destination_dataset=dataset
)

INFO:bigquery_views_manager.materialize_views:materializing view: de_dev.v_data_science_editor_pubmed_ids -> de_dev.mv_data_science_editor_pubmed_ids
INFO:bigquery_views_manager.materialize_views:materialized view: de_dev.v_data_science_editor_pubmed_ids, total rows: 774, 16010972 bytes processed, took: 3.920s
